In [1]:
import sys
print(sys.path)
sys.path.insert(0, "/home/rohanbosworth/manipulation/")



['/home/rohanbosworth/Downloads/iris_and_bins-master', '/home/rohanbosworth/Downloads/iris_and_bins-master/~/manipulation', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/rohanbosworth/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages/IPython/extensions', '/home/rohanbosworth/.ipython']


In [2]:
from pydrake.all import (DiagramBuilder, 
                         AddMultibodyPlantSceneGraph, 
                         Parser, 
                         LoadModelDirectives,
                         ProcessModelDirectives,
                         AddDefaultVisualization,
                         Meldis, PackageMap, RobotDiagramBuilder
)
from pydrake.all import SceneGraphCollisionChecker, VisibilityGraph

from manipulation import ConfigureParser
import os
def plant_builder(use_meshcat = False, use_rohan_scenario = True):
    if use_meshcat:
        meldis = Meldis()
        meshcat = meldis.meshcat
    builder = RobotDiagramBuilder()
    plant = builder.plant()
    scene_graph = builder.scene_graph()
    parser = builder.parser()
    ConfigureParser(parser)
    if use_rohan_scenario:
        directives_file = './models/iiwa14_rohan_cheap.dmd.yaml'
    else:
        directives_file = './models/iiwa14_david_cheap.dmd.yaml'
        
    parser.package_map().AddPackageXml(filename=os.path.abspath("./models/package.xml"))
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    plant.Finalize()
    if use_meshcat:
        visualizer = AddDefaultVisualization(builder.builder(), meshcat)
    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    diagram.ForcedPublish(diagram_context)
    return plant, scene_graph, diagram, diagram_context, plant_context, meshcat if use_meshcat else None


USE_ROHAN = False
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(True, USE_ROHAN)
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
from pydrake.all import (RigidTransform,
                         RotationMatrix,
                         InverseKinematics,
                         Solve,
                         Role,
                         VPolytope,
                         HPolyhedron
                         )
import numpy as np
from tqdm import tqdm

import multiprocessing as mp
from functools import partial


import numpy as np

def point_in_regions(pt, regions):
    """
    check whether point in region
    """
    for r in regions:
        if r.PointInSet(pt.reshape(-1,1)):
            return True
    return False

def sample_in_union_of_polytopes(num_points, regions, aabb_limits, maxit = int(1e4), seed = 1234976512):
    """
    sample in the union of polytopes
    """
    #np.random.seed(seed)
    dim = regions[0].ambient_dimension()
    min_l = aabb_limits[0]
    max_l = aabb_limits[1]
    diff = max_l - min_l
    pts = np.zeros((num_points, dim))
    for i in range(num_points):
        for it in range(maxit):
            pt = min_l + np.random.rand(dim)*diff
            if point_in_regions(pt, regions):
                pts[i,:] = pt
                break
            if it == maxit-1:
                print("[sample_in_union_of_polytopes] NO POINT FOUND")
                return None   
    return pts

def get_cvx_hulls_of_bodies(geometry_names, model_names, plant, scene_graph, scene_graph_context, scaling = 1):
    """
    produce convex hulls of many bodies from plant
    """
    inspector = scene_graph.model_inspector()
    bodies_of_interest = []
    cvx_hulls = []
    for g_n, m_n in zip(geometry_names, model_names):
        b = plant.GetBodyFrameIdIfExists(
                                        plant.GetBodyByName(g_n, 
                                                            plant.GetModelInstanceByName(m_n)
                                                            ).index())
        bodies_of_interest +=[b]
        ids = inspector.GetGeometries(b, Role.kProximity)
        vp = [VPolytope(scene_graph.get_query_output_port().Eval(scene_graph_context), id) for id in ids]
        verts = np.concatenate(tuple([v.vertices().T for v in vp]), axis=0)
        mean = np.mean(verts,axis=0).reshape(1,-1)
        cvx_hulls += [HPolyhedron(VPolytope(scaling*(verts.T- mean.T)+mean.T))]
    return cvx_hulls, bodies_of_interest

from pydrake.all import RotationMatrix, AngleAxis

def sample_random_orientations(N, seed = 1230):
    """
    sample entirely random angles from -pi to pi
    """
    #np.random.seed(seed)
    vecs = np.random.randn(N,3)
    vecs = vecs/np.linalg.norm(vecs)
    angs = 2*np.pi*(np.random.rand(N)-0.5)
    rotmats = [AngleAxis(ang, ax) for ang, ax in zip(angs, vecs)]
    return rotmats

def solve_ik_problem(poses,
                     q0, 
                     frames, 
                     offsets, 
                     plant_ik, 
                     plant_context_ik, 
                     collision_free = True,
                     track_orientation = True,
                     checker = None):
    
    ik = InverseKinematics(plant_ik, plant_context_ik)
    for pose, f, o in zip(poses, frames, offsets):
        ik.AddPositionConstraint(
            f,
            o,
            plant_ik.world_frame(),
            pose.translation()-0.02,
            pose.translation()+0.02,
        )
        if track_orientation:
            ik.AddOrientationConstraint(
                f,
                RotationMatrix(),
                plant_ik.world_frame(),
                pose.rotation(),
                0.1,
            )
    if True:
        ik.AddMinimumDistanceLowerBoundConstraint(0.01, 0.005)
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if result.is_success():
            solution = result.GetSolution(q)
            if checker.CheckConfigCollisionFree(solution):
                return solution
            else:
                print("COLLISION DETECTED")
    return None

def task_space_sampler(num_points_seed_q0_t0_ax_al_tuple, 
                       regions,  
                       plant_builder,
                       frame_names,
                       offsets, 
                       cvx_hulls_of_ROI,
                       ts_min, #bounding box in task space to sample in
                       ts_max,
                       collision_free = True, 
                       track_orientation = True,
                       MAXIT = int(1e4)):
        n_points = num_points_seed_q0_t0_ax_al_tuple[0]
        seed = num_points_seed_q0_t0_ax_al_tuple[1]
        q0 = num_points_seed_q0_t0_ax_al_tuple[2]
        t0 = num_points_seed_q0_t0_ax_al_tuple[3]
        preferred_axis_alignment = num_points_seed_q0_t0_ax_al_tuple[4]
        plant_ik, _, diagram_ik, _, plant_context_ik, _ = plant_builder(use_rohan_scenario=USE_ROHAN)
        frames = [plant_ik.GetFrameByName(f) for f in frame_names]

        robot_instances = [plant_ik.GetModelInstanceByName("iiwa"), plant_ik.GetModelInstanceByName("wsg")]

        checker = SceneGraphCollisionChecker(model = diagram_ik, 
                    robot_model_instances = robot_instances,
                    edge_step_size = 0.125)
    
        q_points = [q0]
        t_points = [t0]
        np.random.seed(seed)    
        if preferred_axis_alignment is not None:
            sc = np.array([0.2, 1, 1])
           
        for i in tqdm(range(n_points)):
            for it in range(MAXIT):
                if preferred_axis_alignment is not None:
                    vecs = np.random.randn(1,3)*sc
                    vecs = vecs/np.linalg.norm(vecs)
                    angs = 1.2*(np.random.randn(1))
                    if preferred_axis_alignment ==2:
                        rot_corr = RotationMatrix.MakeXRotation(-np.pi/2)
                    # if preferred_axis_alignment == 1:
                    #     rot_corr = RotationMatrix.MakeYRotation(np.pi/2)
                    if preferred_axis_alignment == 0:
                        rot_corr = RotationMatrix.MakeZRotation(-np.pi/2)
                    rand_rot = RotationMatrix(AngleAxis(angs[0], vecs[0,:]) )
                    rotmat =  rot_corr@rand_rot
                else:  
                    rotmat = sample_random_orientations(1)[0]
                t_point = sample_in_union_of_polytopes(1, cvx_hulls_of_ROI, [ts_min, ts_max]).squeeze() #t_min + t_diff*np.random.rand(3)
                idx_closest = np.argmin(np.linalg.norm(np.array(t_points)-t_point))
                q0 = q_points[idx_closest]
                res = solve_ik_problem([RigidTransform(rotmat, t_point)], 
                                       q0= q0,
                                       plant_ik=plant_ik,
                                       plant_context_ik=plant_context_ik,
                                       frames=frames,
                                       offsets=offsets,
                                       collision_free = collision_free,
                                       track_orientation =track_orientation,
                                       checker = checker)
                if res is not None and not point_in_regions(res, regions):
                    q_points.append(res)
                    t_points.append(t_point)
                    #print(f"found point {i} seed {seed}")
                    break
                #else:
                #    print(f"failed seed {seed}")
                if it ==MAXIT:
                    print("[SAMPLER] CANT FIND IK SOLUTION")
                    return None, None, True
        return np.array(q_points[1:]), np.array(t_points[1:]), False    

def task_space_sampler_mp(n_points, 
                          regions,  
                          plant_builder,
                          frame_names,
                          offsets,
                          cvx_hulls_of_ROI,
                          ts_min,
                          ts_max, 
                          q0 = None, 
                          t0 = None,
                          collision_free = True, 
                          track_orientation = True,
                          axis_alignment = None
                          ):
        
        processes = mp.cpu_count()
        pool = mp.Pool(processes=processes)
        pieces = np.array_split(np.ones(n_points), processes)
        if q0 is not None:
            n_chunks = [[int(np.sum(p)), np.random.randint(1000), q0, t0, axis_alignment] for p in pieces]
        else:
             plant_ik, _, _, _, plant_context_ik, _ = plant_builder(use_rohan_scenario=USE_ROHAN)
             qmax = plant_ik.GetPositionUpperLimits()
             qmin = plant_ik.GetPositionLowerLimits()
             dim = len(qmax)
             qdiff =qmax- qmin
             chunks = []
             for p in pieces:
                q0 = qdiff*np.random.rand(dim) +qmin
                plant_ik.SetPositions(plant_context_ik, q0)
                t0 = plant_ik.EvalBodyPoseInWorld(plant_context_ik,  plant_ik.GetBodyByName(frame_names[0])).translation()   
                chunks.append([int(np.sum(p)), np.random.randint(1000), q0, t0])
        q_pts = []
        t_pts = []
        is_full = False
        SAMPLERHANDLE = partial(task_space_sampler, 
                                regions = regions, 
                                plant_builder = plant_builder,
                                frame_names = frame_names, 
                                offsets = offsets,
                                cvx_hulls_of_ROI = cvx_hulls_of_ROI,
                                ts_min = ts_min, #bounding box in task space to sample in
                                ts_max = ts_max,
                                collision_free = collision_free,
                                track_orientation = track_orientation) 
        #print(n_chunks)
        results = pool.map(SAMPLERHANDLE, n_chunks)
        for r in results:
            if len(r[0]):
                q_pts.append(r[0])
                t_pts.append(r[1])
                is_full |= r[2]
        return np.concatenate(tuple(q_pts), axis = 0), np.concatenate(tuple(t_pts), axis = 0), is_full, results


In [4]:
import random
import colorsys
from fractions import Fraction
import itertools
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


def hex_to_rgb(hex_color):
    """
    color stuff
    """
    # Remove the leading '#' if present
    hex_color = hex_color.lstrip('#')

    # Ensure the input is a valid hex color code
    if len(hex_color) != 6:
        raise ValueError("Invalid hex color code")

    # Extract the individual color components
    red = int(hex_color[0:2], 16)/255.0
    green = int(hex_color[2:4], 16)/255.0
    blue = int(hex_color[4:6], 16)/255.0

    return red, green, blue

def generate_distinct_colors(n, rgb = False):
    """
    many colors
    """
    cmap = plt.cm.get_cmap('hsv', n)  # Choose a colormap
    colors = [mcolors.rgb2hex(cmap(i)[:3]) for i in range(n)]  # Convert colormap to hexadecimal colors
    if rgb:
        return [hex_to_rgb(c) for c in colors]
    else:
        return colors
    



/home/rohanbosworth/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [5]:
import numpy as np
from pydrake.all import (Hyperellipsoid,
                         MathematicalProgram,
                         Solve, 
                         le,
                         ge,
                         eq,
                         MosekSolver,
                         VPolytope,
                         PiecewisePolynomial,
                         SolverOptions,
                         CommonSolverOption,
                         HPolyhedron)
import pydrake.symbolic as sym
def get_AABB_limits(hpoly, dim=3):
    max_limits = []
    min_limits = []
    A = hpoly.A()
    b = hpoly.b()

    for idx in range(dim):
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(dim, 'x')
        cost = x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A @ x, b))
       
        result = Solve(aabbprog)
        min_limits.append(result.get_optimal_cost() - 0.1)
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(dim, 'x')
        cost = -x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A @ x, b))
        result = Solve(aabbprog)
        max_limits.append(-result.get_optimal_cost() + 0.1)
    return max_limits, min_limits


def get_AABB_cvxhull(regions):
    vps = [VPolytope(r).vertices().T for r in regions]
    cvxh = HPolyhedron(VPolytope(np.concatenate(tuple(vps), axis=0).T))
    max, min = get_AABB_limits(cvxh, dim = 3)    
    return np.array(min), np.array(max), cvxh

In [6]:
from pydrake.all import Rgba, TriangleSurfaceMesh, SurfaceTriangle
from scipy.spatial import ConvexHull

def plot_hpoly3d_2(meshcat, name, hpoly, color, wireframe = True, resolution = -1, offset = np.zeros(3)):
        #meshcat wierdness of double rendering
        hpoly = HPolyhedron(hpoly.A(), hpoly.b() + 0.05*(np.random.rand(hpoly.b().shape[0])-0.5))
        verts = VPolytope(hpoly).vertices().T
        hull = ConvexHull(verts)
        triangles = []
        for s in hull.simplices:
            triangles.append(s)
        tri_drake = [SurfaceTriangle(*t) for t in triangles]
        # obj = self[name]
        # objwf = self[name+'wf']
        # col = to_hex(color)
        #material = MeshLambertMaterial(color=col, opacity=opacity)
        color2 = Rgba(0.8*color.r(), 0.8*color.g(), 0.8*color.b(), color.a())
        meshcat.SetObject(name, TriangleSurfaceMesh(tri_drake, verts+offset.reshape(-1,3)),
                                color, wireframe=False)
        meshcat.SetObject(name+'wf', TriangleSurfaceMesh(tri_drake, verts+offset.reshape(-1,3)),
                                color2, wireframe=True)
        # #obj.set_object(TriangularMeshGeometry(verts, triangles), material)
        # material = MeshLambertMaterial(color=col, opacity=0.95, wireframe=True)
        # objwf.set_object(TriangularMeshGeometry(verts, triangles), material)

def plot_regions(meshcat, regions, ellipses = None,
                     region_suffix = '', colors = None,
                     wireframe = False,
                     opacity = 0.7,
                     fill = True,
                     line_width = 10,
                     darken_factor = .2,
                     el_opacity = 0.3,
                     resolution = 30,
                     offset = np.zeros(3)):
        if colors is None:
            colors = generate_distinct_colors(len(regions))

        for i, region in enumerate(regions):
            c = Rgba(*[col for col in colors[i]],opacity)
            prefix = f"/iris/regions{region_suffix}/{i}"
            name = prefix + "/hpoly"
            if region.ambient_dimension() == 3:
                # plot_hpoly3d(meshcat, name, region,
                #                   c, wireframe = wireframe, resolution = resolution, offset = offset)
                plot_hpoly3d_2(meshcat, name, region,
                                  c, wireframe = wireframe, resolution = resolution, offset = offset)

In [7]:
from pydrake.all import Sphere
col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 

def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/col",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/col",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/col",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

In [8]:

import ipywidgets as widgets
q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=2.9373894, min=-2.9373894, step=0.001)

FloatSlider(value=0.0, description='q1', max=2.0734559999999997, min=-2.0734559999999997, step=0.001)

FloatSlider(value=0.0, description='q2', max=2.9373894, min=-2.9373894, step=0.001)

FloatSlider(value=0.0, description='q3', max=2.0734559999999997, min=-2.0734559999999997, step=0.001)

FloatSlider(value=0.0, description='q4', max=2.9373894, min=-2.9373894, step=0.001)

FloatSlider(value=0.0, description='q5', max=2.0734559999999997, min=-2.0734559999999997, step=0.001)

FloatSlider(value=0.0, description='q6', max=3.0237867, min=-3.0237867, step=0.001)

In [9]:
q

array([-0.0003894, -0.000456 , -0.0003894, -0.000456 , -0.0003894,
       -0.000456 ,  0.0002133])

In [10]:

offset_size = 0.01
geom_names = ['bin_base', 
              'bin_base', 
              'shelves_body',
              ]
model_names = ['binL', 
               'binR', 
               'shelves',
               ]
default_pos = [np.array([-1.37338913e+00,  3.33548849e-01, -1.89389131e-01, -1.50545115e+00,
       -3.89131106e-04,  1.23654885e+00,  2.17070920e-04]),
               np.array([ 9.46108689e-02,  3.33548849e-01, -1.89389131e-01, -1.50545115e+00,
       -3.89131106e-04,  1.23654885e+00,  2.17070920e-04]),
       np.array([ 9.46108689e-02,  3.33548849e-01, -1.89389131e-01, -1.50545115e+00,
       -3.89131106e-04,  1.23654885e+00,  2.17070920e-04]),
       np.array([ 9.46108689e-02,  3.33548849e-01, -1.89389131e-01, -1.50545115e+00,
       -3.89131106e-04,  1.23654885e+00,  2.17070920e-04]),
       np.array([ 9.46108689e-02,  3.33548849e-01, -1.89389131e-01, -1.50545115e+00,
       -3.89131106e-04,  1.23654885e+00,  2.17070920e-04]),
       np.array([ 9.46108689e-02,  3.33548849e-01, -1.89389131e-01, -1.50545115e+00,
       -3.89131106e-04,  1.23654885e+00,  2.17070920e-04]),
       np.array([ 9.46108689e-02,  3.33548849e-01, -1.89389131e-01, -1.50545115e+00,
       -3.89131106e-04,  1.23654885e+00,  2.17070920e-04]),
               
               ]
approach_dir = [2, 
                2, 
                2,
                2, 2, 2, 2
                ] 
approach_sign = [1,1,1, 1, 1, 1]
ts_samplers = []
cols = generate_distinct_colors(2*len(model_names), rgb = True)[1:]
AABB_sampling_regions = []
vertBods = []
for i, (g, m) in enumerate(zip(geom_names, model_names)):
    cvx_hulls_of_ROI_unsc, bodies = get_cvx_hulls_of_bodies([g], [m], plant, scene_graph, scene_graph_context, scaling = 1.0)
    vertsA = [VPolytope(c).vertices().T for c in cvx_hulls_of_ROI_unsc]
    vertBods.append(vertsA[0])
    cvx_hulls_of_ROI = cvx_hulls_of_ROI_unsc

vertBodsNp = np.array(vertBods)

if vertBods[2].any():
    vertShelves = vertBodsNp[-1]
    vertBodsNp = vertBodsNp[:-1]
    vertBods = vertBods[:-1]
    minz = np.min(vertShelves[:,2])
    maxz = np.max(vertShelves[:,2])
    incrVal = (abs(minz) + abs(maxz))/3
    for j in range(3):
        addVerts = vertShelves.copy()
        for i in range(len(addVerts)):
            if addVerts[i][2] == minz:
                addVerts[i][2] = minz + incrVal * j
            if addVerts[i][2] == maxz:
                addVerts[i][2] = minz + incrVal * (j+1)
        
        vertBods.append(addVerts)

for i, verts in enumerate(vertBods):
    cvx_hulls_of_ROI = []
    verts = [verts]
    for v in verts:
        offset = approach_sign[i]*(np.sign(v[:,approach_dir[i]] - np.mean(v[:,approach_dir[i]]))==approach_sign[i])*offset_size
        v[:,approach_dir[i]] += offset #scale*(v[:,approach_dir[i]] - np.mean(v[:,approach_dir[i]])) +  np.mean(v[:,approach_dir[i]])
        cvx_hulls_of_ROI.append(HPolyhedron(VPolytope(v.T)))
    ts_min, ts_max, cvxh_hpoly = get_AABB_cvxhull(cvx_hulls_of_ROI)
    AABB_sampling_regions.append([ts_min, ts_max])
    plot_regions(meshcat, cvx_hulls_of_ROI, region_suffix='',opacity=0.2, colors=[cols[i]])
    q0  = default_pos[i] #np.zeros(7)
    plant.SetPositions(plant_context, q0)
    plant.ForcedPublish(plant_context)
    showres(q0)
    t0 = plant.EvalBodyPoseInWorld(plant_context,  plant.GetBodyByName("body")).translation()       
    sample_handle_ts = partial(task_space_sampler_mp,
                            q0 = q0,
                            t0 = t0,
                            plant_builder = plant_builder,
                            frame_names = ['body'],
                            offsets = [np.array([0,0.1,0])],
                            cvx_hulls_of_ROI =cvx_hulls_of_ROI,
                            ts_min = ts_min,
                            ts_max = ts_max,
                            collision_free = True,
                            track_orientation = True,
                            axis_alignment = None #approach_dir[i]
                            )
    ts_samplers.append(sample_handle_ts)

/tmp/ipykernel_2270808/2708891333.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('hsv', n)  # Choose a colormap


In [11]:
robot_instances = [plant.GetModelInstanceByName("iiwa"), plant.GetModelInstanceByName("wsg")]
checker = SceneGraphCollisionChecker(model = diagram, 
                    robot_model_instances = robot_instances,
                    edge_step_size = 0.125)

def vgraph(points, checker, parallelize):
    ad_mat = VisibilityGraph(checker.Clone(), np.array(points).T, parallelize = parallelize)
    N = ad_mat.shape[0]
    for i in range(N):
        ad_mat[i,i] = False
    #TODO: need to make dense for now to avoid wierd nx bugs for saving the metis file.
    return  ad_mat
vgraph_handle = partial(vgraph, checker = checker, parallelize = True) 

INFO:drake:Allocating contexts to support implicit context parallelism 20


In [12]:
from pydrake.all import MaxCliqueSolverViaGreedy

def compute_double_greedy_clique_partition(adj_mat, min_cliuqe_size, worklimit =100):
    cliques = []
    done = False
    adj_curr = adj_mat.copy()
    #adj_curr = 1- adj_curr
    #np.fill_diagonal(adj_curr, 0)
    for i in range(adj_curr.shape[0]):
        adj_curr[i,i] = False
    ind_curr = np.arange(adj_curr.shape[0])
    solver = MaxCliqueSolverViaGreedy()
    while not done:
        #val, ind_max_clique_local = solve_max_independent_set_integer(adj_curr, worklimit=worklimit) #solve_max_independet_set_KAMIS(adj_curr, maxtime = 5) #
        maximal_clique_bools = solver.SolveMaxClique(adj_curr)
        ind_max_clique_local = np.where(maximal_clique_bools)[0]
        #non_max_ind_local = np.arange(len(adj_curr))
        #non_max_ind_local = np.delete(non_max_ind_local, ind_max_clique_local, None)
        index_max_clique_global = np.array([ind_curr[i] for i in ind_max_clique_local])
        cliques.append(index_max_clique_global.reshape(-1))
        adj_curr = np.delete(adj_curr, ind_max_clique_local, 0)
        adj_curr = np.delete(adj_curr, ind_max_clique_local, 1)
        ind_curr = np.delete(ind_curr, ind_max_clique_local)
        if len(adj_curr) == 0 or len(cliques[-1])<min_cliuqe_size:
            done = True
    return cliques

In [13]:
import os 
import pickle

Npts = 10
seed = 5
if f"SAVVA_{Npts}_{offset_size}_{seed}.pkl" in os.listdir('./tmp'):
    with open(f"./tmp/7DOFBINS_{Npts}_{offset_size}_{seed}.pkl", 'rb') as f:
        d = pickle.load(f)
        q_obj = d['q_obj']
        t_obj = d['t_obj']
        q_tot = d['q_tot']
        t_tot = d['t_tot']        
        ad_mat = d['ad_obj']
        ad_tot = d['ad_tot']
        cliques_obj = d['cliques_obj']
        cliques_tot = d['cliques_tot']
else:
    q_obj = []
    t_obj = []
    ad_obj = []
    cliques_obj = []
    for sh in ts_samplers:
        q, t, _, res = sh(Npts,[])
        ad_mat = vgraph_handle(q)
        q_obj +=[q]
        t_obj +=[t]
        ad_obj +=[ad_mat]
        cliques_obj += [compute_double_greedy_clique_partition(adj_mat=ad_mat.toarray(), min_cliuqe_size=10)]
    t_tot = np.concatenate(tuple(t_obj))
    q_tot = np.concatenate(tuple(q_obj))
    ad_tot = vgraph_handle(np.concatenate(tuple(q_obj), axis= 0))
    #cliques_tot = compute_greedy_clique_partition(ad_tot.toarray(), min_cliuqe_size=10)
    cliques_tot = compute_double_greedy_clique_partition(ad_tot.toarray(), min_cliuqe_size=10)
    with open(f"./tmp/7DOFBINS_{Npts}_{offset_size}_{seed}.pkl", 'wb') as f:
        pickle.dump({'q_obj':q_obj, 
                     't_obj':t_obj, 
                     't_tot': t_tot, 
                     'q_tot': q_tot, 
                     'ad_obj': ad_obj, 
                     'ad_tot' : ad_tot,
                     'cliques_obj': cliques_obj, 
                     'cliques_tot': cliques_tot}, f)

INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Allocating contexts to support implicit context parallelism 20
  0%|          | 0/1 [00:00<?, ?it/s]INFO:drake:Allocating contexts to support implicit context parallelism 20
I

In [14]:
def stretch_array_to_3d(arr, val=0.):
    if arr.shape[0] < 3:
        arr = np.append(arr, val * np.ones((3 - arr.shape[0])))
    return arr
def plot_point(point, meshcat_instance, name,
               color=Rgba(0.06, 0.0, 0, 1), radius=0.01):
    meshcat_instance.SetObject(name,
                               Sphere(radius),
                               color)
    meshcat_instance.SetTransform(name, RigidTransform(
        RotationMatrix(), stretch_array_to_3d(point)))
    

def plot_points(meshcat, points, name, size = 0.05, color = Rgba(0.06, 0.0, 0, 1)):
    for i, pt in enumerate(points):
        n_i = name+f"/pt{i}"
        plot_point(pt, meshcat, n_i, color = color, radius=size)

In [15]:
for i, pts in enumerate(t_obj):
    plot_points(meshcat, pts, f'task_space_pts{i}/')

In [23]:
import time
time.sleep(0)
print(len(q_obj))
for q in q_obj[4]:
    showres(q)
    time.sleep(0.5)

5


In [16]:
#construct regions in bins
ellipsoids_obj = []
for q_bin, clique_decomp in zip(q_obj, cliques_obj):
    ellipsoids_bin = []
    for cl in clique_decomp:
        if len(cl)>=8:
            points_clique = q_bin[cl]
            ellipsoid = Hyperellipsoid.MinimumVolumeCircumscribedEllipsoid(points_clique.T)
            ellipsoids_bin.append(ellipsoid)
    ellipsoids_obj.append(ellipsoids_bin)

#run iris on ellipsoids



In [17]:
cliques_obj

[[array([1, 3])], [array([0])]]